In [18]:
from itertools import chain

def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Wort'].to_list()))
    else:
        vocab = list(set(data['Attribut'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok

In [19]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical

def get_pad_train_test_val(data_group, data):
    n_token = len(list(set(data['Wort'].to_list())))
    n_tag = len(list(set(data['Attribut'].to_list())))

    tokens = data_group['Word_idx'].to_list()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value=n_token-1)

    tags = data_group['Tag_idx'].to_list()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value=tag2idx["O"])

    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntest_tokens length: ', len(test_tokens),
        '\ntrain_tags length: ', len(train_tags),
        '\ntest_tags length: ', len(test_tags)
    )

    return train_tokens, test_tokens, train_tags, test_tags

In [20]:
## class weights berechnen

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)
    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))

In [21]:
import pandas as pd


df = pd.read_csv('Daten/trainingData300.csv',escapechar="\\",sep=",",error_bad_lines=False,warn_bad_lines=False)
#print(len(df))

token2idx, idx2token = get_dict_map(df, 'token')
tag2idx, idx2tag = get_dict_map(df, 'tag')

df['Word_idx'] = df['Wort'].map(token2idx)
df['Tag_idx'] = df['Attribut'].map(tag2idx)
print(df)

df_group = df.groupby(by = ['satzId'], as_index=False)['Wort', 'Attribut', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))
print(df_group)
train_tokens, test_tokens, train_tags, test_tags = get_pad_train_test_val(df_group, df)

         satzId           Wort       Attribut  Word_idx  Tag_idx
0             0          Broan        B-Brand     27322        2
1             0  TEN136WWBroan              O     21752        4
2             0       TEN136WW  B-Modelnumber     82916        6
3             0       Overview              O     54579        4
4             0            The              O     34588        4
...         ...            ...            ...       ...      ...
4928479   55339            AND              O     65913        4
4928480   55339    MAINTENANCE              O     27239        4
4928481   55339       Lockable              O     18760        4
4928482   55339        control              O     34174        4
4928483   55339          panel              O     76106        4

[4928484 rows x 5 columns]
<ipython-input-21-68cf3901bb79>:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_group = df.groupby(by = ['

In [22]:
print(train_tags[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
# von den traing_tags die class weigths berechnen

class_weights = generate_class_weights(train_tags[0], multi_class=True, one_hot_encoded=True)
print(class_weights)

{1: 42.857142857142854, 2: 42.857142857142854, 3: 42.857142857142854, 4: 0.1457725947521866, 5: 42.857142857142854, 6: 42.857142857142854, 7: 42.857142857142854}


In [24]:

input_dim = len(list(set(df['Wort'].to_list())))+1
output_dim = 32
input_length = max([len(s) for s in df_group['Word_idx'].tolist()])
n_tags = len(tag2idx)
print('input_dim: ', input_dim, '\noutput_dim: ', output_dim, '\ninput_length: ', input_length, '\nn_tags: ', n_tags)

input_dim:  103349 
output_dim:  32 
input_length:  300 
n_tags:  8


In [25]:
from numpy.random import seed
import tensorflow
seed(1)
tensorflow.random.set_seed(2)

In [26]:
import numpy as np
#import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

def get_bilstm_lstm_model():
    model = Sequential()

    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))

    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))

    model.add(TimeDistributed(Dense(n_tags, activation="softmax")))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'], sample_weight_mode='temporal')
    model.summary()

    return model

In [35]:
# ToDo: Array korreckt befüllen:
# https://github.com/keras-team/keras/issues/3653#issuecomment-761085597
def generate_sample_weights(train_tags, class_weights): 
    #replaces values for up to 3 classes with the values from class_weights#
    # ToDo: Train Tags sind OnHot Encodings 
    sample_weights = [np.where(y==0,class_weights[0],
                        np.where(y==1,class_weights[1],
                        np.where(y==2,class_weights[2],
                        np.where(y==3,class_weights[3],
                        np.where(y==4,class_weights[4],
                        np.where(y==3,class_weights[5],
                        np.where(y==4,class_weights[6],
                        np.where(y==5,class_weights[7])))))))) for y in training_data]
    return np.asarray(sample_weights)

class_weights = {0: 0.1, 1:300.0, 2:300.0, 3:300.0, 4: 300.0, 5:300.0}

In [36]:
class_weights = np.zeros((39010, 300))
# ToDo: Array korreckt befüllen:
# https://github.com/keras-team/keras/issues/3653#issuecomment-761085597
class_weights[:, 0] += 0.1
class_weights[:, 1] += 300.0
class_weights[:, 2] += 300.0
class_weights[:, 3] += 300.0
class_weights[:, 4] += 300.0
class_weights[:, 5] += 300.0
class_weights[:, 6] += 300.0
class_weights[:, 7] += 300.0

def train_model(X, y, model):
    loss  = list()
    for _ in range(1):
        hist = model.fit(X, y, batch_size=256, verbose=1, epochs=1, validation_split=0.2, sample_weight=class_weights)
        loss.append(hist.history['loss'][0])
    return loss

In [37]:
model_bilstm_lstm = get_bilstm_lstm_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 32)           3307168   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 300, 64)           16640     
_________________________________________________________________
lstm_5 (LSTM)                (None, 300, 32)           12416     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 300, 8)            264       
Total params: 3,336,488
Trainable params: 3,336,488
Non-trainable params: 0
_________________________________________________________________


In [38]:
results = pd.DataFrame()
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

110/110 [==============================] - 852s 8s/step - loss: 5.5693 - accuracy: 0.9754 - val_loss: 3.5370 - val_accuracy: 0.9896


In [39]:
from tensorflow.keras.models import Sequential, save_model, load_model
import numpy as np
import tensorflow as tf
from tensorflow import keras
# Save the model
filepath = './models/train300wWeights'
# save_model(md, filepath)
model_bilstm_lstm.save(filepath)

INFO:tensorflow:Assets written to: ./models/train300wWeights\assets
